Things one can focus at or pipeline sketch:

1. [Build] Creating a robust and easy-implementable paraphrasing pipeline.

2. [Modify] Creating a way to balance unbalanced datasets using paraphrasing (involves detection of imbalance)

3. [Test]: Challenge existing text data augmentation methods

In [1]:
import pandas as pd

# defining examples dataset
original = [
    "Danmark kan ikke leve af gæld.",
    "Efter et regeringsskifte vil den økonomiske politik bygge på et meget klart forsigtighedsprincip.",
    "Vi vil oprette et agentur mod falske nyheder, der udarbejder modsvar og korrigerer falske nyhedshistorier i Danmark.",
    "Lige netop sygeskatten kunne have været et udspil, som LO og DA kunne have brugt selv til at få forhandlingerne på gled igen - alt i alt et indgreb og en lovgivning, der er tydeligt præget af at være udarbejdet af politikere fra Overdanmark med funktionærverdenen som eneste erfaringsgrundlag og uden særlig meget viden om, hvor skoen trykker.",
    "HK, har ønsket at få kønsopdelt lønstatistik.",
    "Så jeg synes, det er ret svært, når man skal igennem de her ting, og jeg synes heller ikke, vi kan være bekendt, at noget, der er så vigtigt som forbrugernes sikkerhed, når det gælder medicin, skal puttes ind sammen med en hel masse andre ting.",
    "Grænseoverskridende problemer må løses gennem et grænseoverskridende samarbejde.",
    "Forskellen mellem virkningerne på beskæftigelse og arbejdsløshed ligger i den øgede uddannelsesindsats.",
    "Det er sådan, at jeg lige har været inde at tjekke min ordførertale, så jeg ikke skulle få samme svada fra hr.",
    "Det ene er at give borgerne mulighed for frit at kunne rejse rundt i Europa, så de f.eks. ikke skal stå i lange køer, når de skal på ferie nede sydpå.",
    "Derfor har vi også i den seneste folketingsperiode givet forsvaret det substantielle løft, som vi lovede, at der ville komme.",
    "Dansk Folkeparti har her fremsat et beslutningsforslag, der pålægger regeringen at arbejde for en ændring af reglerne for EF-Domstolens arbejde, og så vidt jeg ved, skal det drøftes den 13. marts.",
    "Vi skal være bedre til at få nye idéer omsat til produktion.",
    "Vi har selvfølgelig noteret os, at der ikke er flertal for vores forslag.",
    "Ved at starte en forbudsprocedure med udgangspunkt i listen over uønskede stoffer og de hormonforstyrrende kategori 1-stoffer i kosmetik tages et vigtigt skridt i den rigtige retning.",
    "Mænd og kvinder og i øvrigt alle andre grupper skal have lige muligheder for at udfylde deres personlige potentiale.",
    "Undervejs har vi truffet hårde beslutninger.",
    "Regeringen magtede ikke at fortsætte en politik, som kunne forbedre erhvervslivets forhold.",
    "Derfor undrer det mig, at der er så meget tvivl, og at man kan tro, at vi vil tage 20 pct. af de unge mennesker ind i den her ordning.",
    "Så det går da trods alt, også i forhold til tallene fra sidste år, den rigtige vej."
]

paraphrased = [
    "Danmark kan ikke overleve ved konstant at låne penge.",
    "Sker der et skift i regeringen, vil den nye økonomiske politik grundlæggende være baseret på en omhyggelig forsigtighed.",
    "Vi vil danne en organisation til bekæmpelse af fake news, som vil udvikle svar og rette op på urigtige nyheder i Danmark.",
    "Sygeskatten kunne have været noget, som LO og DA kunne have brugt til selv at sætte gang i forhandlingerne igen - i sidste ende en politisk handling og lovgivning, der tydeligvis er skabt af politikere fra den bedre del af samfundet, kun baseret på erfaringer fra administrativt arbejde og med begrænset indsigt i de konkrete problemer.",
    "HK har udtrykt ønske om statistikker over lønninger opdelt efter køn.",
    "Det er meget svært at håndtere disse problemstillinger, og jeg finder det også uacceptabelt, at forbrugernes medicinsikkerhed skal blandes sammen med en række andre mindre relevante sager.",
    "Problemer, der går på tværs af landegrænser, skal adresseres gennem internationalt samarbejde.",
    "Forskellen i påvirkningen af beskæftigelse og arbejdsløshed skyldes den forstærkede indsats i uddannelse.",
    "Jeg har lige sikret mig, at jeg har tjekket min taler for at undgå at blive mødt med den samme monolog fra hr. [person].",
    "Det handler om at give borgerne frihed til at rejse rundt i Europa uden at opleve lange ventetider, for eksempel ved grænsekontroller på ferier i syden.",
    "Derfor har vi i den seneste valgperiode givet militæret den betydelige forstærkning, vi lovede ville finde sted.",
    "Dansk Folkeparti har stillet et forslag, som forpligter regeringen til at arbejde for ændrede regler vedrørende EU-Domstolens funktion, og dette skal efter planen diskuteres den 13. marts.",
    "Vi skal blive bedre til at transformere nye ideer til faktisk produktion.",
    "Vi har selvfølgelig bemærket, at der ikke er opbakning til vores forslag.",
    "Det er et væsentligt skridt i den rigtige retning at indlede en forbudsproces baseret på listen over bannede stoffer og hormonforstyrrende stoffer i kategori 1 i kosmetiske produkter.",
    "Mænd og kvinder, samt alle andre, skal have lige adgang til at realisere deres personlige potentiale.",
    "Undervejs har vi truffet nogle vanskelige valg.",
    "Regeringen havde ikke styrken til at fortsætte med en politik, der kunne forbedre vilkårene for erhvervslivet.",
    "Derfor undrer det mig, at der er så stor usikkerhed, og at nogen tror, vi vil inkludere 20 % af de unge i denne ordning.",
    "Så på trods af alt tyder det i forhold til sidste års tal på, at vi er på rette vej."
]

examples = pd.DataFrame({"original": original, "paraphrased": paraphrased})

In [2]:
# before running this, run the following in the terminal: 
#1# CT_METAL=1 pip install ctransformers --no-binary ctransformers
#2# pip install huggingface-hub
#3# huggingface-cli download TheBloke/OpenHermes-2.5-Mistral-7B-GGUF openhermes-2.5-mistral-7b.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False


# playing with mistral
#from data_load import load_loop, map_questions
from ctransformers import AutoModelForCausalLM, AutoConfig
from pathlib import Path

def make_input_mistral(phrase: str) -> str:
    system = f"""Du er en sprogmodel som forstår og taler kompetent dansk.
    Du svarer kun på dansk.
    Din opgave er at omformulere tekst ved at finde synonymer med samme betydning.
    Her er nogle eksempler på omformuleringer:
    {examples.original[0]} -> {examples.paraphrased[0]},
    {examples.original[1]} -> {examples.paraphrased[1]},
    {examples.original[2]} -> {examples.paraphrased[2]},
    {examples.original[3]} -> {examples.paraphrased[3]},
    {examples.original[4]} -> {examples.paraphrased[4]},
    {examples.original[5]} -> {examples.paraphrased[5]},
    Her er nogle regler du skal overholde:
    Du må ikke gentage dig selv.
    Du må ikke bruge samme sætning som i originalen.
    Du skal bruge samme kontekst som i originalen.
    """

    prompt = f"""
    <|im_start|>system
    {system}<|im_end|>
    <|im_start|>user
    {phrase}<|im_end|>
    <|im_start|>assistant
    """

    return prompt

def load_mistral(model_path):
    #config = AutoConfig.from_pretrained(model_path)
    # Explicitly set the max_seq_len
    #config.max_seq_len = 4096
    #config.max_answer_len= 1024

    model = AutoModelForCausalLM.from_pretrained(
        str(model_path), 
        model_type="mistral",
        gpu_layers=50,
        temperature=0.8, # default is 0.8
        top_p = 0.95,
        top_k = 40,  # default is 0.95
        max_new_tokens = 1000,
        context_length = 6000)
    
    return model

from tqdm import tqdm

def generate_dataframe(original, model): # original = list of strings, model = output from the load_mistral() function
    data = []
    for string in tqdm(original):
        new = model(make_input_mistral(string))
        data.append([string, new])
    df = pd.DataFrame(data, columns=['Original', 'New'])
    return df


/Users/wibe/Desktop/CogSci/NLP/NLP_exam_2023/venv_NLP_exam/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/wibe/Desktop/CogSci/NLP/NLP_exam_2023/venv_NLP_exam/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os


max_new_tokens = 50

model = load_mistral(os.path.join("model", "openhermes-2.5-mistral-7b.Q4_K_M.gguf"))

strings = ["Partiet Venstre er et borgerligt parti.",
           "Social demokraterne var i regering fra 2011 til 2015.",
           "Radikale venstre synes at skatten var for høj."]

#for string in tqdm(original):
#    new = model(make_input_mistral(string))
#    print(f'[Original] {string} || [New] {new}')


df = generate_dataframe(original, model)



 55%|█████▌    | 11/20 [01:28<00:49,  5.51s/it]

# 1. Paraphrasing pipeline

Token

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("berkeley-nest/Starling-LM-7B-alpha")
model = AutoModelForCausalLM.from_pretrained("berkeley-nest/Starling-LM-7B-alpha")

/Users/wibe/Desktop/CogSci/NLP/Exam_project/NLP_exam_2023/venv_NLP_exam/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 1.61k/1.61k [00:00<00:00, 601kB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 1.46MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:01<00:00, 1.50MB/s]
added_tokens.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 96.4kB/s]
special_tokens_map.json: 100%|██████████| 560/560 [00:00<00:00, 1.21MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
config.json: 100%|██████████| 613/613 [00:00<00:00, 1.26MB/s]
model.safetensors.index.json: 100%|██████████| 23.9k/23.9k [00:00<00:00, 14.3MB/s]
model-00001-of-00003.safetensors:   7%|▋         | 346M/4.94G [02:51<38:01, 

KeyboardInterrupt: 

In [4]:
import openai
import os

openai.api_key = os.environ["OPENAI_API_KEY"]

# show version of openai
print(openai.__version__)


0.27.7


In [2]:
message = """
1.	Danmark kan ikke leve af gæld.
2.	Efter et regeringsskifte vil den økonomiske politik bygge på et meget klart forsigtighedsprincip.
3.	Vi vil oprette et agentur mod falske nyheder, der udarbejder modsvar og korrigerer falske nyhedshistorier i Danmark.
4.	Lige netop sygeskatten kunne have været et udspil, som LO og DA kunne have brugt selv til at få forhandlingerne på gled igen - alt i alt et indgreb og en lovgivning, der er tydeligt præget af at være udarbejdet af politikere fra Overdanmark med funktionærverdenen som eneste erfaringsgrundlag og uden særlig meget viden om, hvor skoen trykker.
5.	HK har ønsket at få kønsopdelt lønstatistik.
6.	Så jeg synes, det er ret svært, når man skal igennem de her ting, og jeg synes heller ikke, vi kan være bekendt, at noget, der er så vigtigt som forbrugernes sikkerhed, når det gælder medicin, skal puttes ind sammen med en hel masse andre ting.
7.	Grænseoverskridende problemer må løses gennem et grænseoverskridende samarbejde.
8.	Forskellen mellem virkningerne på beskæftigelse og arbejdsløshed ligger i den øgede uddannelsesindsats.
9.	Det er sådan, at jeg lige har været inde at tjekke min ordførertale, så jeg ikke skulle få samme svada fra hr.
10.	Det ene er at give borgerne mulighed for frit at kunne rejse rundt i Europa, så de f.eks. ikke skal stå i lange køer, når de skal på ferie nede sydpå.
11.	Derfor har vi også i den seneste folketingsperiode givet forsvaret det substantielle løft, som vi lovede, at der ville komme.
12.	Dansk Folkeparti har her fremsat et beslutningsforslag, der pålægger regeringen at arbejde for en ændring af reglerne for EF-Domstolens arbejde, og så vidt jeg ved, skal det drøftes den 13. marts.
13.	Vi skal være bedre til at få nye idéer omsat til produktion.
14.	Vi har selvfølgelig noteret os, at der ikke er flertal for vores forslag.
15.	Ved at starte en forbudsprocedure med udgangspunkt i listen over uønskede stoffer og de hormonforstyrrende kategori 1-stoffer i kosmetik tages et vigtigt skridt i den rigtige retning.
16.	Mænd og kvinder og i øvrigt alle andre grupper skal have lige muligheder for at udfylde deres personlige potentiale.
17.	Undervejs har vi truffet hårde beslutninger.
18.	Regeringen magtede ikke at fortsætte en politik, som kunne forbedre erhvervslivets forhold.
19.	Derfor undrer det mig, at der er så meget tvivl, og at man kan tro, at vi vil tage 20 pct. af de unge mennesker ind i den her ordning.
20.	Så det går da trods alt, også i forhold til tallene fra sidste år, den rigtige vej.
"""


In [ ]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
import openai

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "Paraphrase the following sentence (in Danish): "},
        {"role": "user", "content": message},
    ]
)

In [5]:
# show completed chat
response['choices'][0]['message']['content']

'1. Danmark kan ikke overleve ved at have gæld.\n2. Efter et regeringsskifte vil den økonomiske politik være baseret på et klart forsigtighedsprincip.\n3. Vi vil etablere en agentur mod falske nyheder, som vil udarbejde modsvar og rette falske nyhedshistorier i Danmark.\n4. Netop sygeskat kunne have været et udspil, som LO og DA selv kunne have brugt til at få forhandlingerne på sporet igen - alt i alt en indgriben og lovgivning, som tydeligvis er udviklet af politikere fra Overdanmark med kun funktionsverdenen som deres erfaringsgrundlag og uden særlig meget viden om, hvor problemet ligger.\n5. HK har ønsket at få opdelt lønstatistik efter køn.\n6. Så jeg synes, det er ret svært, når man skal igennem alt dette, og jeg synes heller ikke, at det er forsvarligt, at noget, der er så vigtigt som forbrugernes sikkerhed vedrørende medicin, bliver bundtet sammen med en masse andre ting.\n7. Problemer på tværs af grænser skal løses gennem et internationalt samarbejde.\n8. Forskellen mellem eff

In [5]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "This is example sentence"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
#print(embeddings)

# print cosine-similarities betwen the sentences
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([embeddings[0]], [embeddings[1]])

array([[0.987843]], dtype=float32)

In [1]:
import nltk

def check_sentence_syntax(sentence):
    # Tokenize the sentence into individual words
    words = nltk.word_tokenize(sentence)

    # Perform part-of-speech tagging on the words
    tagged_words = nltk.pos_tag(words)

    # Check for any syntax errors
    syntax_errors = []
    for word, tag in tagged_words:
        if tag.startswith('NN') and word.lower() != 'syntax':
            syntax_errors.append(word)

    return syntax_errors

sentence = "The cat sat on the mat."
syntax_errors = check_sentence_syntax(sentence)
if len(syntax_errors) > 0:
    print("Syntax errors found:", syntax_errors)
else:
    print("No syntax errors found.")


LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - '/Users/wibe/nltk_data'
    - '/Users/wibe/Desktop/CogSci/NLP/Exam_project/NLP_exam_2023/venv_NLP_exam/nltk_data'
    - '/Users/wibe/Desktop/CogSci/NLP/Exam_project/NLP_exam_2023/venv_NLP_exam/share/nltk_data'
    - '/Users/wibe/Desktop/CogSci/NLP/Exam_project/NLP_exam_2023/venv_NLP_exam/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
